In [7]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import numpy as np
from linear_cca import linear_cca
from torch.utils.data import BatchSampler, SequentialSampler
from DeepCCAModels import DeepCCA
from main import Solver
from utils import load_data, svm_classify
try:
    import cPickle as thepickle
except ImportError:
    import _pickle as thepickle

import gzip
import numpy as np
torch.set_default_tensor_type(torch.DoubleTensor)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
############
# Parameters Section

device = torch.device('cuda')
print("Using", torch.cuda.device_count(), "GPUs")

# the path to save the final learned features
save_to = './new_features.gz'

# the size of the new space learned by the model (number of the new features)
outdim_size = 15

# size of the input for view 1 and view 2
input_shape1 = 784
input_shape2 = 784

# number of layers with nodes in each one
layer_sizes1 = [1024, 1024, 1024, outdim_size]
layer_sizes2 = [1024, 1024, 1024, outdim_size]

# the parameters for training the network
learning_rate = 1e-3
epoch_num = 15
batch_size = 800

# the regularization parameter of the network
# seems necessary to avoid the gradient exploding especially when non-saturating activations are used
reg_par = 1e-5

# specifies if all the singular values should get used to calculate the correlation or just the top outdim_size ones
# if one option does not work for a network or dataset, try the other one
use_all_singular_values = False

# if a linear CCA should get applied on the learned features extracted from the networks
# it does not affect the performance on noisy MNIST significantly
apply_linear_cca = True
# end of parameters section
############

# Each view is stored in a gzip file separately. They will get downloaded the first time the code gets executed.
# Datasets get stored under the datasets folder of user's Keras folder
# normally under [Home Folder]/.keras/datasets/


Using 0 GPUs


In [9]:
data1 = load_data('./noisymnist_view1.gz')
data2 = load_data('./noisymnist_view2.gz')

loading data ...
loading data ...


In [29]:
data1[0][0][0].view(28, 28).numpy()

array([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000

In [120]:
import matplotlib.pyplot as plt
%matplotlib inline

x = torch.tensor([c0,c1,c2,c3,c4])
plt.imsave('../images/mnist/data1.pdf', x.view(28*5, 28*5).numpy().transpose() , cmap='gray')

In [118]:

c0 = np.concatenate((data1[0][0][0].view(28, 28).numpy(), 
                     data1[0][0][1].view(28, 28).numpy(), 
                     data1[0][0][2].view(28, 28).numpy(), 
                     data1[0][0][4].view(28, 28).numpy(), 
                     data1[0][0][5].view(28, 28).numpy()), 
                    axis=1)
c1 = np.concatenate((data1[0][0][6].view(28, 28).numpy(), 
                     data1[0][0][7].view(28, 28).numpy(), 
                     data1[0][0][8].view(28, 28).numpy(), 
                     data1[0][0][9].view(28, 28).numpy(), 
                     data1[0][0][10].view(28, 28).numpy()), 
                    axis=1)
c2 = np.concatenate((data1[0][0][11].view(28, 28).numpy(), 
                     data1[0][0][12].view(28, 28).numpy(), 
                     data1[0][0][13].view(28, 28).numpy(), 
                     data1[0][0][14].view(28, 28).numpy(), 
                     data1[0][0][15].view(28, 28).numpy()), 
                    axis=1)
c3 = np.concatenate((data1[0][0][16].view(28, 28).numpy(), 
                     data1[0][0][17].view(28, 28).numpy(), 
                     data1[0][0][18].view(28, 28).numpy(), 
                     data1[0][0][19].view(28, 28).numpy(), 
                     data1[0][0][20].view(28, 28).numpy()), 
                    axis=1)
c4 = np.concatenate((data1[0][0][21].view(28, 28).numpy(), 
                     data1[0][0][22].view(28, 28).numpy(), 
                     data1[0][0][23].view(28, 28).numpy(), 
                     data1[0][0][24].view(28, 28).numpy(), 
                     data1[0][0][25].view(28, 28).numpy()), 
                    axis=1)

In [116]:
c0 = data1[0][0][0].view(28, 28).numpy()
for j in range(9):
    c0 = np.concatenate((c0, data1[0][0][i+1].view(28, 28).numpy()), axis=1)

In [5]:
# Building, training, and producing the new features by DCCA
model = DeepCCA(layer_sizes1, layer_sizes2, input_shape1,
                input_shape2, outdim_size, use_all_singular_values, device=device).double()
l_cca = None
if apply_linear_cca:
    l_cca = linear_cca()
solver = Solver(model, l_cca, outdim_size, epoch_num, batch_size,
                learning_rate, reg_par, device=device)
train1, train2 = data1[0][0], data2[0][0]
val1, val2 = data1[1][0], data2[1][0]
test1, test2 = data1[2][0], data2[2][0]
# val1=None
# test1=None
solver.fit(train1, train2, val1, val2, test1, test2)
# TODO: Save linear_cca model if needed

# set_size = [0, train1.size(0), train1.size(
#     0) + val1.size(0), train1.size(0) + val1.size(0) + test1.size(0)]
# loss, outputs = solver.test(torch.cat([train1, val1, test1], dim=0), torch.cat(
#     [train2, val2, test2], dim=0), apply_linear_cca)

AssertionError: 
The NVIDIA driver on your system is too old (found version 9020).
Please update your GPU driver by downloading and installing a new
version from the URL: http://www.nvidia.com/Download/index.aspx
Alternatively, go to: https://pytorch.org to install
a PyTorch version that has been compiled with your version
of the CUDA driver.

In [ ]:
set_size = [0, train1.size(0), train1.size(
    0) + val1.size(0), train1.size(0) + val1.size(0) + test1.size(0)]
loss, outputs = solver.test(torch.cat([train1, val1, test1], dim=0), torch.cat(
    [train2, val2, test2], dim=0), apply_linear_cca)

In [ ]:
l_cca.fit(np.array(train1), np.array(train2), 15)

In [ ]:
v1, v2 = l_cca.test(np.array(torch.cat([train1, val1, test1], dim=0)), np.array(torch.cat([train2, val2, test2], dim=0)))

In [ ]:
new_data = []
for idx in range(3):
    new_data.append([outputs[0][set_size[idx]:set_size[idx + 1], :], data1[idx][1]])
# Training and testing of SVM with linear kernel on the view 1 with new features
[test_acc, valid_acc] = svm_classify(new_data, C=0.01)
print("Accuracy on view 1 (validation data) is:", valid_acc * 100.0)
print("Accuracy on view 1 (test data) is:", test_acc*100.0)

In [ ]:
new_data = []
for idx in range(3):
    new_data.append([v1[set_size[idx]:set_size[idx + 1], :], data1[idx][1]])
# Training and testing of SVM with linear kernel on the view 1 with new features
[test_acc, valid_acc] = svm_classify(new_data, C=0.01)
print("Accuracy on view 1 (validation data) is:", valid_acc * 100.0)
print("Accuracy on view 1 (test data) is:", test_acc*100.0)

In [ ]:
new_data = []
for idx in range(3):
    new_data.append([np.array(torch.cat([train1, val1, test1], dim=0))[set_size[idx]:set_size[idx + 1], :], data1[idx][1]])
# Training and testing of SVM with linear kernel on the view 1 with new features
[test_acc, valid_acc] = svm_classify(new_data, C=0.01)
print("Accuracy on view 1 (validation data) is:", valid_acc * 100.0)
print("Accuracy on view 1 (test data) is:", test_acc*100.0)

In [ ]:
np.shape(np.array(torch.cat([train1, val1, test1], dim=0)))

In [ ]:
d = torch.load('checkpoint.model')
solver.model.load_state_dict(d)
solver.model.parameters()

In [6]:
# Saving new features in a gzip pickled file specified by save_to
print('saving new features ...')
f1 = gzip.open(save_to, 'wb')
thepickle.dump(new_data, f1)
f1.close()

saving new features ...


NameError: name 'new_data' is not defined